# 回帰モデルの構築: データの準備と可視化

## **カボチャの線形回帰 - レッスン2**
#### はじめに

TidymodelsとTidyverseを使って機械学習モデルの構築を始める準備が整った今、データに対して質問を投げかける準備ができました。データを扱い、機械学習のソリューションを適用する際には、適切な質問をすることでデータセットの可能性を最大限に引き出すことが非常に重要です。

このレッスンでは以下を学びます:

- モデル構築のためのデータ準備方法。

- `ggplot2`を使ったデータの可視化方法。

解きたい質問によって使用する機械学習アルゴリズムの種類が決まります。そして、得られる答えの質はデータの性質に大きく依存します。

これを実際の演習を通じて確認してみましょう。


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>イラスト: @allison_horst</figcaption>


<!--![イラスト: \@allison_horst](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.ja.jpg)<br>イラスト: \@allison_horst-->


## 1. かぼちゃデータのインポートとTidyverseの召喚

このレッスンを効率よく進めるために、以下のパッケージが必要です：

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) は、データサイエンスをより速く、簡単に、そして楽しくするために設計された[Rパッケージのコレクション](https://www.tidyverse.org/packages)です！

以下のコマンドでインストールできます：

`install.packages(c("tidyverse"))`

以下のスクリプトは、このモジュールを完了するために必要なパッケージがインストールされているか確認し、足りない場合は自動的にインストールします。


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

では、このレッスン用に提供された[データ](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv)を読み込み、いくつかのパッケージを起動しましょう！


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

`glimpse()` を使うと、空白があり、文字列（`chr`）と数値データ（`dbl`）が混在していることがすぐにわかります。`Date` は文字型で、さらに `Package` という奇妙な列があり、データが `sacks` や `bins`、その他の値が混在しています。実際、このデータは少し混乱していますね 😤。

実際のところ、完全に整ったデータセットをそのまま使って機械学習モデルを作成できることはあまりありません。でも心配しないでください。このレッスンでは、標準的なRライブラリを使って生のデータセットを準備する方法を学びます 🧑‍🔧。さらに、データを視覚化するためのさまざまなテクニックも学びます。📈📊
<br>

> おさらい: パイプ演算子（`%>%`）は、オブジェクトを関数や呼び出し式に順次渡すことで、論理的な順序で操作を実行します。コード内でパイプ演算子は「そして次に」と言っているようなものだと考えることができます。


## 2. データの欠損を確認する

データサイエンティストがよく直面する問題の一つに、不完全または欠損しているデータがあります。Rでは、欠損値や不明な値を特別なセントネル値 `NA` (Not Available) で表します。

では、データフレームに欠損値が含まれているかどうかをどうやって確認するのでしょうか？
<br>
-   一つの簡単な方法は、ベースRの関数 `anyNA` を使用することです。この関数は論理オブジェクト `TRUE` または `FALSE` を返します。


In [ ]:
pumpkins %>% 
  anyNA()

素晴らしいですね、いくつかのデータが欠けているようです！そこから始めるのが良いでしょう。

-   別の方法として、`is.na()`関数を使用することができます。この関数は、各列の要素が欠けているかどうかを論理値`TRUE`で示します。


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

大きなデータフレームの場合、このようにすべての行と列を個別に確認するのは非効率的で、実質的に不可能です😴。

-   より直感的な方法としては、各列の欠損値の合計を計算することです：


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

データが欠けている部分はありますが、今回の作業には影響がないかもしれません。さらに分析を進めてみましょう。

> 素晴らしいパッケージや関数のセットに加えて、Rには非常に優れたドキュメントがあります。例えば、`help(colSums)` または `?colSums` を使用して、その関数について詳しく調べることができます。


## 3. Dplyr: データ操作の文法

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>@allison_horstによるイラスト</figcaption>


<!--![@allison_horstによるイラスト](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.ja.png)<br/>@allison_horstによるイラスト-->


[`dplyr`](https://dplyr.tidyverse.org/)は、Tidyverseのパッケージであり、データ操作の文法を提供します。これにより、最も一般的なデータ操作の課題を解決するための一貫した動詞のセットが利用できます。このセクションでは、dplyrの動詞について詳しく見ていきましょう！  


#### dplyr::select()

`select()`は、`dplyr`パッケージに含まれる関数で、保持したい列や除外したい列を選択するのに役立ちます。

データフレームを扱いやすくするために、`select()`を使って必要な列だけを残し、いくつかの列を削除しましょう。

例えば、この演習では、分析に必要な列として`Package`、`Low Price`、`High Price`、`Date`を使用します。これらの列を選択してみましょう。


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()`は、`dplyr`パッケージの関数で、既存の列を保持しながら、新しい列を作成したり、既存の列を変更したりするのに役立ちます。

`mutate`の一般的な構文は以下の通りです：

`data %>%   mutate(new_column_name = what_it_contains)`

`Date`列を使って`mutate`を試してみましょう。以下の操作を行います：

1.  日付（現在は文字型）を月の形式に変換します（これらはアメリカの日付なので、形式は`MM/DD/YYYY`です）。

2.  日付から月を抽出し、新しい列に保存します。

Rでは、[lubridate](https://lubridate.tidyverse.org/)パッケージを使うと、日付データや日時データの操作が簡単になります。そこで、`dplyr::mutate()`、`lubridate::mdy()`、`lubridate::month()`を使って、上記の目的を達成してみましょう。なお、以降の操作で`Date`列は不要になるため、削除しても問題ありません。


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

やったね！🤩

次に、新しい列 `Price` を作成しましょう。この列はカボチャの平均価格を表します。では、`Low Price` 列と `High Price` 列の平均を取り、新しい `Price` 列に値を埋めていきましょう。
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

イエス！💪

「でも待って！」と、`View(pumpkins)`でデータセット全体をざっと見た後に言うかもしれません。「ここ、何かおかしくない？」🤔

`Package`列を見てみると、カボチャはさまざまな形態で販売されていることがわかります。一部は`1 1/9 bushel`の単位で、一部は`1/2 bushel`の単位で、一部はカボチャ1個ごと、一部はポンドごと、さらに幅の異なる大きな箱で販売されているものもあります。

これを確認してみましょう：


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

素晴らしいですね！👏

カボチャは一貫して重さを測るのが非常に難しいようですので、`Package`列に*bushel*という文字列が含まれるカボチャだけを選択し、新しいデータフレーム`new_pumpkins`に入れましょう。


#### dplyr::filter() と stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): 条件を満たす**行**のみを含むデータのサブセットを作成します。この場合、`Package`列に文字列 *bushel* を含むかどうかでカボチャをフィルタリングします。

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): 文字列内で特定のパターンが存在するかどうかを検出します。

[`stringr`](https://github.com/tidyverse/stringr) パッケージは、一般的な文字列操作を簡単に行うための関数を提供します。


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

私たちは、バスケット単位でカボチャを含む約415行のデータに絞り込んだことがわかります。🤩  
<br>


#### dplyr::case_when()

**でも、もう一つやるべきことがあります**

バスケットの量が行ごとに異なることに気づきましたか？価格を1 1/9や1/2バスケットではなく、1バスケットあたりの価格として正規化する必要があります。標準化するために少し計算をしましょう。

[`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) 関数を使って、条件に応じてPrice列を*変換*します。`case_when`を使うと、複数の`if_else()`ステートメントをベクトル化することができます。


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

さて、私たちはバスケット単位での価格分析を行うことができます。しかし、このカボチャのバスケットに関する研究は、「データの性質を理解すること」がいかに`重要`であるかを示しています！

> ✅ [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308)によると、バスケットの重さはその中身の種類によって異なります。これは体積の測定方法だからです。「例えば、トマトのバスケットは56ポンドの重さがあるとされています... 葉物や緑の野菜はスペースを多く取り、重さが少ないため、ほうれん草のバスケットはたった20ポンドです。」かなり複雑ですね！バスケットからポンドへの換算はやめて、バスケット単位で価格を設定しましょう。このカボチャのバスケットに関する研究は、データの性質を理解することがいかに重要であるかを示しています！

> ✅ 半バスケット単位で販売されているカボチャが非常に高価であることに気づきましたか？その理由を考えられますか？ヒント：小さなカボチャは大きなカボチャよりもずっと高価です。おそらく、大きな空洞のあるパイ用カボチャが1つ占めるスペースに比べて、小さなカボチャがバスケットにたくさん詰め込まれるためでしょう。


さて、最後に、ちょっとした冒険のために 💁‍♀️、Month列を最初の位置、つまり`Package`列の「前」に移動させましょう。

列の位置を変更するには、`dplyr::relocate()`を使用します。


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

よくできました！👌 これで、新しい回帰モデルを構築するためのクリーンで整然としたデータセットが手に入りましたね！  
<br>


## 4. ggplot2を使ったデータ可視化

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>ダサニ・マディパリによるインフォグラフィック</figcaption>


<!--![ダサニ・マディパリによるインフォグラフィック](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.ja.png){width="600"}-->

こんな*賢い*言葉があります：

> 「シンプルなグラフは、他のどんな手段よりも多くの情報をデータアナリストの頭に届けてくれる。」 --- ジョン・テューキー

データサイエンティストの役割の一部は、扱っているデータの質や性質を示すことです。そのために、彼らはしばしば興味深い可視化、つまりプロットやグラフ、チャートを作成し、データのさまざまな側面を示します。このようにして、視覚的に関係性やギャップを示すことができ、通常では見つけにくいものを明らかにすることができます。

可視化は、データに最も適した機械学習手法を決定するのにも役立ちます。例えば、散布図が直線に沿っているように見える場合、そのデータは線形回帰の演習に適していることを示しています。

Rにはグラフを作成するためのいくつかのシステムがありますが、[`ggplot2`](https://ggplot2.tidyverse.org/index.html)はその中でも最もエレガントで汎用性の高いものの一つです。`ggplot2`を使うと、**独立したコンポーネントを組み合わせる**ことでグラフを作成することができます。

まずは、Price列とMonth列を使ったシンプルな散布図から始めてみましょう。

この場合、まず[`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html)を使い、データセットと美的マッピング（[`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)を使用）を指定し、その後、散布図用のレイヤー（例えば[`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)）を追加します。


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

このプロットは役に立つと思いますか 🤷？何か驚く点はありますか？

特に役に立つわけではありません。単にデータを特定の月に点として散らばらせて表示しているだけです。
<br>


### **どうすれば役立つものにできるのか？**

チャートに有益なデータを表示させるには、通常データを何らかの方法でグループ化する必要があります。例えば、私たちの場合、各月ごとのカボチャの平均価格を見つけることで、データの基礎的なパターンについてより深い洞察を得ることができます。このことから、もう一つの**dplyr**の便利な機能にたどり着きます：

#### `dplyr::group_by() %>% summarize()`

Rでのグループ化集計は以下の方法で簡単に計算できます：

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()`は、分析単位をデータセット全体から月ごとなどの個別のグループに変更します。

-   `dplyr::summarize()`は、グループ化変数ごとに1つの列、指定した集計統計ごとに1つの列を持つ新しいデータフレームを作成します。

例えば、`dplyr::group_by() %>% summarize()`を使用して、**Month**列に基づいてカボチャをグループ化し、各月の**平均価格**を求めることができます。


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

簡潔に！✨

月のようなカテゴリカルな特徴は、棒グラフ📊で表現するのが適しています。棒グラフを作成するためのレイヤーは `geom_bar()` と `geom_col()` です。詳細を知りたい場合は `?geom_bar` を参照してください。

さあ、作ってみましょう！


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩これは、より役立つデータビジュアライゼーションですね！9月と10月にカボチャの価格が最も高くなることを示しているようです。これはあなたの予想に合っていますか？なぜそう思いますか、またはなぜそうではないですか？

第2レッスンの完了、おめでとうございます👏！データをモデル構築のために準備し、その後、ビジュアライゼーションを使ってさらに多くの洞察を得ることができましたね！



---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当社は責任を負いません。
